In [ ]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install urllib3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 745 kB/s 
     |████████████████████████████████| 251 kB 14.5 MB/s 


In [ ]:
#Train arrival history

import urllib.request as ul
from bs4 import BeautifulSoup as soup
import datetime

stationDeparture = 'London+St+Pancras+International+(STP)'
stationArrival = 'Hendon+(HEN)'
date = '31/08/2022' #DDMMYYYY

url = 'https://www.recenttraintimes.co.uk/Home/Search?Op=Srch&Fr=' + stationDeparture + '&To=' + stationArrival +'&TimTyp=D&TimDay=A&Days=Al&TimPer=Cu&dtFr=' + date + '&dtTo=' + date + '&ShwTim=AvAr&MxArCl=5&TOC=All&ArrSta=5&MetAvg=Mea&MetSpr=RT&MxScDu=&MxSvAg=&MnScCt='
req = ul.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
client = ul.urlopen(req)
htmldata = client.read()
client.close()

pagesoup = soup(htmldata, "html.parser")
itemlocator = pagesoup.findAll('table')

timetable = itemlocator[1]
trainlocator = timetable.findAll('tr')

counter = 0
for train in trainlocator:
  if counter < 2:
    counter = counter + 1
  else: 
    if counter == 2:
      counter = 0
    trainInfo = train.findAll('td')
    operator = trainInfo[0].text
    scheduledDeparture = trainInfo[1].text
    scheduledArrival = trainInfo[2].text
    duration = trainInfo[3].text
    realArrival = trainInfo[5].text.split()[0]

    try:
      startTime = datetime.datetime.strptime(scheduledArrival.split(':')[0] + ":" + scheduledArrival.split(':')[1] + ":00", "%H:%M:%S")
    except:
       startTime = datetime.datetime.strptime(scheduledArrival.replace('*','').split(':')[0] + ":" + scheduledArrival.replace('*','').split(':')[1] + ":00", "%H:%M:%S")
    endTime = datetime.datetime.strptime(realArrival.split(':')[0] + ":" + realArrival.split(':')[1] + ":00", "%H:%M:%S")
    diffTime = endTime - startTime
    min = int(diffTime.total_seconds() / 60)
    if min == 0:
      late = " (0 mins late)"
    if min < 0:
      midTime = datetime.datetime.strptime("00:00:00","%H:%M:%S")
      timeChange = datetime.timedelta(minutes=1440)
      diffTime = (endTime - midTime) + (midTime - startTime + timeChange)
      min = int(diffTime.total_seconds() / 60)
      late = " (" + str(min) + " mins late)"
    else:
      late = " (" + str(min) + " mins late)"

    info = operator + ' from ' + stationDeparture.replace('+', ' ') + ' to ' + stationArrival.replace('+', ' ') + ' ' + scheduledDeparture + ' -> ' + scheduledArrival + ' ' + duration + ' Arrived at ' + realArrival + late
    print(info)

TL from London St Pancras International (STP) to Hendon (HEN) 00:18 -> 00:34 16m  Arrived at 00:36 (2 mins late)
TL from London St Pancras International (STP) to Hendon (HEN) 02:48 -> 03:05 17m  Arrived at 03:20 (15 mins late)
TL from London St Pancras International (STP) to Hendon (HEN) 05:28 -> 05:43 15m  Arrived at 05:43 (0 mins late)
TL from London St Pancras International (STP) to Hendon (HEN) 06:43 -> 06:58 15m  Arrived at 06:58 (0 mins late)
TL from London St Pancras International (STP) to Hendon (HEN) 07:28 -> 07:44 16m  Arrived at 07:59 (15 mins late)
TL from London St Pancras International (STP) to Hendon (HEN) 08:13 -> 08:28 15m  Arrived at 08:28 (0 mins late)
TL from London St Pancras International (STP) to Hendon (HEN) 09:18 -> 09:35 17m  Arrived at 09:35 (0 mins late)
TL from London St Pancras International (STP) to Hendon (HEN) 09:48 -> 10:05 17m  Arrived at 10:05 (0 mins late)
TL from London St Pancras International (STP) to Hendon (HEN) 10:33 -> 10:50 17m  Arrived at 1

In [ ]:
#Station code
import urllib.request as ul
from bs4 import BeautifulSoup as soup

stationName = "london liv"
firstCharacter = stationName[0].upper()
code = ""

url = 'https://en.wikipedia.org/wiki/UK_railway_stations_%E2%80%93_' + firstCharacter
req = ul.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
client = ul.urlopen(req)
htmldata = client.read()
client.close()

pagesoup = soup(htmldata, "html.parser")
itemlocator = pagesoup.findAll('table')
stationLocator = itemlocator[1]

stationList = stationLocator.findAll('tr')
counter = 0
for i in stationList:
  if counter == 0:
    counter = counter + 1
  else:
    info = i.findAll('a')
    name = info[0].text
    if name.lower().startswith(stationName.lower()):
       print(info[0].text)
       code = info[2].text
       break

print(code)

London Liverpool Street
LST


In [ ]:
#Real time arrival XXX

import urllib.request as ul
from bs4 import BeautifulSoup as soup

stationCode = "RDG"
url = 'https://ojp.nationalrail.co.uk/service/ldbboard/dep/' + stationCode.upper()
req = ul.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
client = ul.urlopen(req)
htmldata = client.read()
client.close()

pagesoup = soup(htmldata, "html.parser")
itemlocator = pagesoup.findAll('table')
departureBoard = itemlocator[0]

serviceLocator = departureBoard.tbody.findAll('tr')
for i in serviceLocator:
  infoLocator = i.findAll('td')
  time = infoLocator[0].text
  destination = infoLocator[1].text.replace("\n",'').lstrip()
  destination = " ".join(destination.split())
  status = infoLocator[2].text
  status = " ".join(status.split())
  platform = infoLocator[3].text
  if platform == "":
    print(time + ' to ' + destination + ' ' + status)
  else:
    print(time + ' to ' + destination + ' ' + status + ': Platform ' + platform)

17:27 to Bristol Temple Meads 17:52 25 mins late: Platform 8
17:29 to Penzance 17:55 26 mins late: Platform 7
17:41 to London Paddington 17:47 6 mins late: Platform 10
17:41 to Newbury 
17:41 to Newbury Delayed: Platform 8
17:43 to London Paddington 17:45 2 mins late: Platform 11A
17:43 to Swansea Delayed: Platform 9
17:48 to Ealing Broadway On time: Platform 14
17:50 to Gatwick Airport via Guildford On time: Platform 5
17:50 to London Paddington 17:51 1 mins late: Platform 10
17:52 to Bournemouth On time: Platform 7
17:57 to Basingstoke On time: Platform 2
17:57 to Taunton Delayed: Platform 8
17:58 to Didcot Parkway Delayed: Platform 12B
17:58 to Ealing Broadway On time: Platform 14
17:58 to London Paddington 18:05 7 mins late: Platform 11
18:00 to Hereford Delayed: Platform 9
18:00 to London Paddington On time: Platform 10A
18:03 to Plymouth Delayed: Platform 7
18:07 to Newbury On time: Platform 1
18:08 to Ealing Broadway On time: Platform 15A
18:12 to London Paddington On time: Plat